# Pivo Recommender

Looking for a new way to compare and recommend beer.  

This notebook scrapes beer reviews from [BeerAdvocate](https://www.beeradvocate.com/) then performs natural language processing on these reviews.  Once a profile of a beer has been created, a similar, semi-similar, or completely different beer can be recommended.

'/Users/johnzupan/mystuff/pivo_rec'

In [ ]:
%cd beerAdvocateScraper
%scrapy crawl reviewScraper

In [1]:
import itertools as it
import os
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en')

## Phrase Modeling

Phrase modeling is an approach to learning combinations of tokens that together represent meaning multi-word concepts.  These phrase models are developed by looping over the words in the corpus and finding words that appear together more than they should by random chance.  The formula used to determine if whether two tokens $A$ and $B$ constitute a phrase is:

$$
\frac {count(A B) - count_{min}}
{count(A) * count(B)}
*N > threshold
$$

where:
* $count(A)$ is the number of times $A$ appears in the corpus
* $count(B)$ is the number of times $B$ appears in the corpus
* $count(AB)$ is the number of times $AB$ appear in the corpus in that order
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that the phrase appears a minimum number of times
* $threshold$ is a user-defined paramter to control how strong the relationship must be before the two tokens are considered a single concept

Once we have trained the phrase model we can apply it to the reviews in our corpus.  It will consider the multiworded tokens to be single phrases.

The gensim library will help us with phrase modeling, specifically the Phrases class.

In [3]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [3]:
%mv beerAdvocateScraper/beerAdvocateScraper/BeerAdvocateReviews.csv ./intermediate/

In [4]:
import re
pattern = re.compile('^[\d,"]+')

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    count = 0
    with open(filename, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split(pattern, review)[1].replace('\\n', '\n')
            except: pass
            if count % 1000 == 0:
                print(f'on review {count}')
            count += 1
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=1000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [130]:
intermediate_directory = os.path.join('.', 'intermediate')
# review_txt_filepath = os.path.join(intermediate_directory,'BeerAdvocateReviews.csv')
review_txt_filepath = os.path.join(intermediate_directory,'slimReviews.csv')
ngram_all = os.path.join(intermediate_directory, 'ngram_all')
unigram_sentences_filepath = os.path.join(ngram_all,
                                          'unigram_sentences_all.txt')

We will use `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text.  We will write this data back out to a new file (`unigram_sentence_all`), with one normalized sentence per line.  We will use this data for learning our phrase models.

In [131]:
with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        print(sentence)
        f.write(sentence + '\n')

on review 0
black and thick
tan head
nose be perfect with huge maple syrup and coffee
bourbon and vanilla
good breakfast ever
taste maple vanilla bourbon and oak
everything -PRON- want in a beer
not overly roasted
just perfect
full bodied
good carbonation
end with maple syrup
amazing
that be all
can not believe -PRON- live up to the hype
oz
bottle pour into a snifter
no bottle number
appearance inky black
stain the glass again
big and dark head than mornin delight and assassin
great surface wisps
smell bourbon
maple
vanilla
dark chocolate
everything -PRON- would want in a stout
the bourbon do take over though
taste bourbon be sharp
coffee come out here
dark chocolate
maple come out in the back
sweetness linger
bourbon flavor linger also
mouthfeel very thick
boozy
sweet with medium carbonation
overall tough to argue which be good but bourbon always win
serve alongside mornin delight and assassin
the beer be near black with a filmy beige head
all hype and bias aside this aroma be silly g

Our data in the `unigram_sentences_all` file is now organized as a large text file with one sentence per line.  This format allows us to use gensim's LineSentence class, a convenient iterator for working with gensim's other components.  It *streams* the documents/sentences from disk, so you never have to hold the entire corpus in RAM at once.  This allows you to scale the modeling to a very large corpora.

In [132]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [133]:
for unigram_sentence in it.islice(unigram_sentences, 5,7):
    print(u' '.join(unigram_sentence))
    print(u' ')

taste maple vanilla bourbon and oak
 
everything -PRON- want in a beer
 


In [134]:
bigram_model_file = os.path.join(ngram_all, 'bigram_model')

In [135]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [136]:
bigram_sentences_filepath = os.path.join(ngram_all,'bigram_sentences_all.txt')

In [137]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [138]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [139]:
for bigram_sentence in it.islice(bigram_sentences, 5, 7):
    print(u' '.join(bigram_sentence))
    print(u'')

taste maple vanilla bourbon and oak

everything -PRON- want in a beer



In [140]:
trigram_model_filepath = os.path.join(ngram_all, 'trigram_model')

In [141]:
if 1 == 1:

    phrases_trigram = Phrases(bigram_sentences)
    trigram_model = Phraser(phrases_trigram)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [142]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [143]:
trigram_sentences_filepath = os.path.join(ngram_all, 'trigram_sentences_all.txt')

In [144]:
with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [145]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [146]:
for trigram_sentence in it.islice(trigram_sentences, 3, 5):
    print(u' '.join(trigram_sentence))
    print(u'')

bourbon and vanilla

good breakfast ever



In [147]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [148]:
from spacy.lang.en.stop_words import STOP_WORDS

In [149]:

with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in STOP_WORDS]
        
        # remove pronouns
        trigram_review = [term for term in trigram_review
                          if term !='-PRON-']
        

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

on review 0


In [150]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 0, 3):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 0, 3):
        print(review)

Original:

on review 0
Black and thick. Tan head. Nose is perfect with huge maple syrup and coffee. Bourbon and vanilla. Best breakfast ever. Taste: maple vanilla bourbon and oak. Everything I want in a beer. Not overly roasted. Just perfect.Full bodied. Good carbonation. Ends with maple syrup.Amazing. That's all. Cannot believe it lives up to the hype.

 oz. bottle poured into a snifter. No bottle number.Appearance - Inky black. Stains the glass again. Bigger and darker head than Mornin' Delight and Assassin. Great surface wisps.Smell - Bourbon. Maple. Vanilla. Dark chocolate. Everything you'd want in a stout. The bourbon does take over, though.Taste - Bourbon is sharp. Coffee comes out here. Dark chocolate. Maple comes out in the back. Sweetness lingers. Bourbon flavor lingers also.Mouthfeel - Very thick. Boozy. Sweet with medium carbonation. Overall - Tough to argue which is better, but bourbon always wins."

Served alongside Mornin' Delight and Assassin. The beer is near black with

Exception ignored in: <generator object line_review at 0x113ac0c50>
RuntimeError: generator ignored GeneratorExit


In [28]:
def get_beerID(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    with open(review_txt_filepath, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split('(^[\d"]+)', review)[1].replace('\\n', '\n')
            except: pass

In [62]:
from collections import Counter

In [151]:
beerDict = {}


for i in range(3):
    with open(trigram_reviews_filepath, encoding='utf_8') as f:
        for review in it.islice(f, i, i+1):
            beerID = list(it.islice(get_beerID(review_txt_filepath), i, i+1))[0]
            wordDict = Counter()
            for word in review.split():
                wordDict[word] = wordDict.get(word,0) + 1
#             print(wordDict)
#             print('---------------')
            beerDict[beerID] = beerDict.get(beerID, Counter()) + wordDict
print(beerDict)

{'78820': Counter({'bourbon': 7, 'vanilla': 4, 'good': 4, 'black': 3, 'head': 3, 'coffee': 3, 'maple': 3, 'thick': 2, 'perfect': 2, 'maple_syrup': 2, 'taste': 2, 'oak': 2, 'want': 2, 'beer': 2, 'carbonation': 2, 'amazing': 2, 'hype': 2, 'bottle': 2, 'mornin_delight': 2, 'assassin': 2, 'dark_chocolate': 2, 'come': 2, 'linger': 2, 'tan': 1, 'nose': 1, 'huge': 1, 'breakfast': 1, 'overly': 1, 'roasted': 1, 'full_bodied': 1, 'end': 1, 'believe': 1, 'live': 1, 'oz': 1, 'pour_into': 1, 'snifter': 1, 'number': 1, 'appearance': 1, 'inky': 1, 'stain': 1, 'glass': 1, 'big': 1, 'dark': 1, 'great': 1, 'surface': 1, 'wisps': 1, 'smell': 1, 'stout': 1, 'sharp': 1, 'sweetness': 1, 'flavor': 1, 'mouthfeel': 1, 'boozy': 1, 'sweet': 1, 'medium': 1, 'overall': 1, 'tough': 1, 'argue': 1, 'win': 1, 'serve': 1, 'alongside': 1, 'near': 1, 'filmy': 1, 'beige': 1, 'bias': 1, 'aside': 1, 'aroma': 1, 'silly': 1, 'roast': 1, 'world': 1, 'class': 1, 'sip': 1, 'definition': 1, 'grace': 1, 'balance': 1, 'silky': 1, '

Exception ignored in: <generator object get_beerID at 0x113579a98>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x1135795c8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113579570>
RuntimeError: generator ignored GeneratorExit


In [152]:
m = pd.read_csv(review_txt_filepath)

beerDict = {}

for i in range(m.shape[0]):
    with open(trigram_reviews_filepath, encoding='utf_8') as f:
        for review in it.islice(f, i, i+1):
            beerID = list(it.islice(get_beerID(review_txt_filepath), i, i+1))[0]
            wordDict = Counter()
            for word in review.split():
                wordDict[word] = wordDict.get(word,0) + 1
#             print(wordDict)
#             print('---------------')
            beerDict[beerID] = beerDict.get(beerID, Counter()) + wordDict
print(beerDict)

{'78820': Counter({'beer': 39, 'coffee': 36, 'bourbon': 32, 'thick': 23, 'like': 22, 'maple_syrup': 21, 'black': 20, 'head': 18, 'dark': 18, 'barrel': 18, 'vanilla': 17, 'good': 17, 'maple': 17, 'taste': 15, 'smell': 15, 'bottle': 14, 'oak': 13, 'carbonation': 13, 'flavor': 12, 'sweet': 12, 'morning_delight': 12, 'perfect': 11, 'pour': 11, 'glass': 10, 'stout': 10, 'medium': 10, 'chocolate': 10, 'hype': 9, 'big': 8, 'dark_chocolate': 8, 'sweetness': 8, 'note': 8, 'brown': 8, 'finish': 8, 'linger': 7, 'mouthfeel': 7, 'aroma': 7, 'world': 7, 'palate': 7, 'molass': 7, 'lot_of': 7, 'mocha': 7, 'malt': 7, 'caramel': 7, 'creamy': 7, 'nose': 6, 'roasted': 6, 'mornin_delight': 6, 'come': 6, 'smooth': 6, 'topple_goliath': 6, 'lace': 6, 'mouth': 6, 'feel': 6, 'slight': 6, 'drink': 6, 'body': 6, 'milk_chocolate': 6, 'do_not': 6, 'whisky': 6, 'full_bodied': 5, 'oz': 5, 'roast': 5, 'viscous': 5, 'complex': 5, 'slightly': 5, 'thank': 5, 'age': 5, 'nice': 5, 'close': 5, 'brownie': 5, 'whiskey': 5, 'h

Exception ignored in: <generator object get_beerID at 0x1135795c8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa7410>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa73b8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113ac0ca8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa7410>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa73b8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113ac0ca8>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa7410>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x113aa73b8>
RuntimeError: generator ignor

In [153]:
beerDict.keys()

dict_keys(['78820', '29619', '263364'])

## Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short is to learn dense numerical representations of each term in a corpus vocabulary.  If the model is succesful, the vectors it learns should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary.  Word vector models are fully unsupervised &mdash; they learn all of these meaning and relationships solely by analyzing the text of the corpus, without any advanced knowledge provided.

In [154]:
from gensim.models import Word2Vec

# run with or without stop words removed and word lemmatized
trigram_sentences = LineSentence(trigram_reviews_filepath)
# trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

In [155]:
if 1==1:
    
    # initiate the model and perform one epoch of training
    beer2vec = Word2Vec(trigram_sentences, size=100, window=5,
                       min_count=1, sg=1) #workers=?
    beer2vec.save(word2vec_filepath)
    
    #perform the next n epochs of training
    for i in range(1, 5):
        beer2vec.train(trigram_sentences, total_examples=beer2vec.corpus_count, epochs=2)
        beer2vec.save(word2vec_filepath)
        
        
#load the finished model from disk
beer2vec = Word2Vec.load(word2vec_filepath)
beer2vec.init_sims()

print(f'Trained model for {beer2vec.train_count} epochs.')

Trained model for 5 epochs.


In [156]:

# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in beer2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda tup: -1*tup[-1])

# # unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# # create a DataFrame with the food2vec vectors as data,
# # and the terms as row labels
word_vectors = pd.DataFrame(beer2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors.head(10)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
beer,-0.125418,-0.158803,-0.109744,-0.088840,-0.050977,-0.001870,0.016279,-0.185949,-0.039158,0.087716,...,-0.078661,0.036378,0.153309,-0.010076,-0.156118,0.132822,-0.133748,0.032135,-0.063877,0.008066
coffee,-0.120252,-0.158009,-0.110976,-0.076152,-0.024895,-0.018940,0.005613,-0.206732,-0.046788,0.077841,...,-0.099884,0.028322,0.142997,-0.020933,-0.173337,0.139913,-0.146807,0.034235,-0.071801,0.024402
good,-0.117596,-0.160508,-0.104429,-0.092543,-0.055181,-0.000042,0.009518,-0.190182,-0.038457,0.089727,...,-0.086296,0.037800,0.153077,-0.013839,-0.165245,0.144309,-0.137846,0.033341,-0.065672,0.014435
bourbon,-0.117719,-0.166992,-0.113476,-0.081326,-0.028596,-0.026475,0.002908,-0.208270,-0.046134,0.072953,...,-0.098826,0.032469,0.139741,-0.026106,-0.176493,0.146143,-0.149110,0.042539,-0.072123,0.026749
like,-0.122866,-0.158534,-0.103148,-0.096458,-0.050301,-0.007634,0.009195,-0.195771,-0.040439,0.083294,...,-0.088329,0.032533,0.149556,-0.018398,-0.166274,0.142531,-0.141494,0.029365,-0.053144,0.014208
taste,-0.119117,-0.157880,-0.097596,-0.091254,-0.043978,-0.005375,-0.000733,-0.201222,-0.035124,0.084128,...,-0.090750,0.039437,0.150297,-0.031229,-0.171644,0.141391,-0.140367,0.046041,-0.063106,0.016063
orange,-0.115265,-0.148698,-0.079513,-0.091739,-0.068124,0.011262,0.004916,-0.211494,-0.028896,0.082787,...,-0.081488,0.029175,0.166646,-0.016828,-0.157962,0.130845,-0.136029,0.048447,-0.060855,0.013049
head,-0.127512,-0.159969,-0.099372,-0.091060,-0.052993,-0.010643,0.013123,-0.207309,-0.045511,0.071138,...,-0.072158,0.023239,0.141622,-0.010100,-0.151235,0.133201,-0.129466,0.021450,-0.051630,0.009956
thick,-0.120823,-0.153335,-0.104436,-0.091314,-0.051827,-0.007216,0.013301,-0.194472,-0.045840,0.085248,...,-0.082513,0.024361,0.150181,-0.016103,-0.158109,0.130717,-0.136189,0.031421,-0.067042,0.006278
carbonation,-0.120230,-0.149243,-0.100329,-0.096964,-0.052994,-0.002225,0.020507,-0.193858,-0.040370,0.088880,...,-0.084265,0.030089,0.152854,-0.016838,-0.165938,0.144155,-0.143085,0.032696,-0.064299,0.006760


In [87]:
word_vectors.shape

(239, 100)

# Dimension Reduction Using t-SNE 

t-Distributed Stochastic Neighbor Embedding, or *t-SNE*, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets.  It attempts to map high-dimensional data onto a low (2 or 3) dimensional  representation such that the relative distance between points are preserved as closely as possible in both high and low dimensional space.

In [157]:
from sklearn.manifold import TSNE

In [161]:
# from spacy.lang.en.stop_words import STOP_WORDS

tsneInput = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsneInput = tsneInput.head(400)

In [163]:
tsneInput.shape

(400, 100)

In [164]:
tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')

In [165]:
import pickle

In [166]:
tsneInput.values.shape

(400, 100)

In [167]:
if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsneInput.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsneInput.index),
                            columns=[u'x_coord', u'y_coord'])

In [168]:
tsne_vectors[u'word'] = tsne_vectors.index

In [169]:
tsne_vectors.head()

,x_coord,y_coord,word
beer,-4.141456,7.934711,beer
coffee,20.921259,-10.803796,coffee
good,-3.244442,-9.479774,good
bourbon,21.402313,-11.091810,bourbon
like,6.317479,-12.092600,like


In [170]:
ids = list(beerDict.keys())
print(ids)

['78820', '29619', '263364']


In [171]:
list1 = []
for i in tsne_vectors.index:
    list1.append(i + '_x')
    list1.append(i + '_y')
print(list1)

['beer_x', 'beer_y', 'coffee_x', 'coffee_y', 'good_x', 'good_y', 'bourbon_x', 'bourbon_y', 'like_x', 'like_y', 'taste_x', 'taste_y', 'orange_x', 'orange_y', 'head_x', 'head_y', 'thick_x', 'thick_y', 'carbonation_x', 'carbonation_y', 'sweet_x', 'sweet_y', 'smell_x', 'smell_y', 'maple_syrup_x', 'maple_syrup_y', 'black_x', 'black_y', 'medium_x', 'medium_y', 'creamy_x', 'creamy_y', 'dark_x', 'dark_y', 'malt_x', 'malt_y', 'grapefruit_x', 'grapefruit_y', 'barrel_x', 'barrel_y', 'nice_x', 'nice_y', 'pour_x', 'pour_y', 'vanilla_x', 'vanilla_y', 'maple_x', 'maple_y', 'glass_x', 'glass_y', 'little_x', 'little_y', 'finish_x', 'finish_y', 'bright_x', 'bright_y', 'nose_x', 'nose_y', 'bottle_x', 'bottle_y', 'feel_x', 'feel_y', 'sweetness_x', 'sweetness_y', 'body_x', 'body_y', 'citrus_x', 'citrus_y', 'oak_x', 'oak_y', 'flavor_x', 'flavor_y', 'note_x', 'note_y', 'lace_x', 'lace_y', 'bitter_x', 'bitter_y', 'light_x', 'light_y', 'perfect_x', 'perfect_y', 'come_x', 'come_y', 'aroma_x', 'aroma_y', 'bit_of

In [174]:
masterDF = pd.DataFrame(columns=list1, index=ids)
masterDF.head()

,beer_x,beer_y,coffee_x,coffee_y,good_x,good_y,bourbon_x,bourbon_y,like_x,like_y,...,gentle_x,gentle_y,hit_x,hit_y,roll_x,roll_y,advertise_x,advertise_y,let_x,let_y
78820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
masterDF

,coffee_x,coffee_y,bourbon_x,bourbon_y,chocolate_x,chocolate_y,beer_x,beer_y,black_x,black_y,...,perfect_x,perfect_y,taste_x,taste_y,hype_x,hype_y,pour_x,pour_y,medium_x,medium_y
78820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.2983,370.203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
for key, value in beerDict.items():
    for k2, v2 in value.items():
        try:
            x_val = tsne_vectors.loc[k2]['x_coord'] * v2
            columnLookupX = k2 + '_x'
            masterDF.loc[key][columnLookupX] = x_val

            y_val = tsne_vectors.loc[k2]['y_coord'] * v2
            columnLookupY = k2 + '_y'
            masterDF.loc[key][columnLookupY] = y_val
        except KeyError:
            pass

In [105]:
tsne_vectors.loc['coffee']['x_coord']

-101.04704

In [176]:
masterDF

,beer_x,beer_y,coffee_x,coffee_y,good_x,good_y,bourbon_x,bourbon_y,like_x,like_y,...,gentle_x,gentle_y,hit_x,hit_y,roll_x,roll_y,advertise_x,advertise_y,let_x,let_y
78820,-161.517,309.454,753.165,-388.937,-55.1555,-161.156,684.874,-354.938,138.985,-266.037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29619,-8.28291,15.8694,NaN,NaN,-12.9778,-37.9191,NaN,NaN,6.31748,-12.0926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263364,-66.2633,126.955,NaN,NaN,-38.9333,-113.757,NaN,NaN,56.8573,-108.833,...,-12.2947,9.69225,17.0956,3.97984,-11.189,7.70579,15.8058,6.6904,13.7154,-17.9538


In [177]:
masterDF.fillna(0, inplace=True)

In [178]:
masterDF

,beer_x,beer_y,coffee_x,coffee_y,good_x,good_y,bourbon_x,bourbon_y,like_x,like_y,...,gentle_x,gentle_y,hit_x,hit_y,roll_x,roll_y,advertise_x,advertise_y,let_x,let_y
78820,-161.516770,309.453728,753.165321,-388.936649,-55.155506,-161.156150,684.874023,-354.937927,138.984530,-266.037197,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
29619,-8.282911,15.869422,0.000000,0.000000,-12.977766,-37.919094,0.000000,0.000000,6.317479,-12.092600,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
263364,-66.263290,126.955376,0.000000,0.000000,-38.933298,-113.757282,0.000000,0.000000,56.857308,-108.833399,...,-12.294682,9.692248,17.095621,3.979839,-11.189038,7.705787,15.805769,6.690396,13.715393,-17.95384


In [195]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from itertools import combinations

In [192]:
list(masterDF.index)

['78820', '29619', '263364']

In [199]:
for pair in combinations(list(masterDF.index), 2):
    cs = 1 - spatial.distance.cosine(masterDF.loc[pair[0]].values.reshape(-1,1), masterDF.loc[pair[1]].values.reshape(-1,1))
    print(f'The similarity between {pair[0]} and {pair[1]} is {cs}')

The similarity between 78820 and 29619 is 0.31598742578451433
The similarity between 78820 and 263364 is 0.27963418099781556
The similarity between 29619 and 263364 is 0.5599509788419919


In [ ]:
# find the similarity between every beer pair
# create a dictionary where the key is the beer and value is the list of beers from most to least similar